## Goal: have a script to run from start to finish that will call ingredients endpoint and upload ingredients to Cosmos DB

In [283]:
# reading in packages
import pandas as pd
import numpy as np
import re
import json
import requests

In [272]:
# reading in databases
food_details = pd.read_excel('raw_data/Release 1 - Food details file.xlsx')

food_nutrients_excel = pd.ExcelFile('raw_data/Release 1 - Food nutrient database.xlsx')
food_nutrients = pd.read_excel(food_nutrients_excel,'All solids & liquids per 100g')

food_measures = pd.read_excel('raw_data/Release 1 - Measures file.xlsx')
food_recipe = pd.read_excel('raw_data/Release 1 - Recipe file.xlsx')

food_emissions = pd.read_excel('raw_data/emissions.xlsx')
food_categories = pd.read_excel('raw_data/categories.xlsx')

initial_classification = pd.read_csv('raw_data/initial_classification.csv')

# to store all of the food product classifications, now we can drop Classification Name from the food database
food_classification = food_details[['Classification ID', 'Classification Name']].copy()
food_classification.drop_duplicates(inplace = True)

# need to fix up the merged columns (ie. make the "Unnamed" column match the previous column name) and remove the units row (make a part of column name)
regex = re.compile(r'Unnamed') # regex to find the "Unnamed" columns

old_columns = list(food_nutrients.columns) # list of existing columns
units = food_nutrients[0:1].values[0] # list of the unit row
new_columns = [] # empty list to store the new column values

for each in range(len(old_columns)):
    if re.search(regex, food_nutrients.columns[each]): # ie. find the column that is "unnamed"
        try:
            new_columns.append(old_columns[each-1] + ' (in ' + units[each] + ')') # take the previous column name and add units
        except:
            new_columns.append(old_columns[each-1]) # if you can't add the units, just keep the column
    else: # ie. column is not "unnamed"
        try:
            new_columns.append(old_columns[each] + ' (in ' + units[each] + ')')
        except:
            new_columns.append(old_columns[each])
    
food_nutrients.rename(columns = dict(zip(old_columns, new_columns)), inplace = True) # rename the columns
food_nutrients.drop(0, axis = 0, inplace = True) # drop the unit row
food_nutrients.reset_index(drop = True, inplace = True) # reset the index

## Information about our datasets
* food_details # information about the ingredient
* food_nutrients # nutritional information about the ingredient
* food_measures # information about the measures of ingredients
* food_recipe # a list of recipes (useless)
* food_emissions # ghg emissions table 
* food_categories # HSR calculation
* food_classification # food classifications which came with the dataset

## Joining on the Initial Classification (so that we can calculate HSR rating and emissions)
HSR calculation and GHG emissions relies on having a particular classification to a group. Since our open data sets don't have these groups, I have had to manually create an initial_classification file, which I have done at a broad category level, and will apply down to the individual ingredients. Not accurate, but quick.

In [273]:
food_details = food_details.merge(initial_classification, on = 'Classification ID', how = 'left')

## Joining nutritional data and food_details so we can calculate HSR

In [274]:
food_details = food_details.merge(food_nutrients, on = 'Public Food Key')

## Calculating HSR for Ingredients
Now that food_details have emission_id, hsr_id as well as all nutritional data we can calculate HSR rating for each ingredient (where available), and also GHG emissions (approximate)

In [275]:
# Reading in hsr tables
hsr_table_1 = pd.read_csv('raw_data/hsr_table_1.csv')
hsr_table_2 = pd.read_csv('raw_data/hsr_table_2.csv')
hsr_table_3 = pd.read_csv('raw_data/hsr_table_3.csv')
hsr_table_4 = pd.read_csv('raw_data/hsr_table_4.csv')

# function to find the closest value in a list
def get_closest_value(arr, target):
    n = len(arr)
    left = 0
    right = n - 1
    mid = 0

    # edge case - last or above all
    if target >= arr[n - 1]:
        return arr[n - 1]
    # edge case - first or below all
    if target <= arr[0]:
        return arr[0]
    # BSearch solution: Time & Space: Log(N)

    while left < right:
        mid = (left + right) // 2  # find the mid
        if target < arr[mid]:
            right = mid
        elif target > arr[mid]:
            left = mid + 1
        else:
            return arr[mid]

    if target < arr[mid]:
        return find_closest(arr[mid - 1], arr[mid], target)
    else:
        return find_closest(arr[mid], arr[mid + 1], target)


# findClosest
# We find the closest by taking the difference
# between the target and both values. It assumes
# that val2 is greater than val1 and target lies
# between these two. 
def find_closest(val1, val2, target):
    return val2 if target - val1 >= val2 - target else val1

# function to find index using get_closest_value
# could probably optimise get_closest_value to just return the index, but maybe later
def index_finder(arr, target):
    if target > get_closest_value(arr, target):
         return arr.index(get_closest_value(arr,target))
    else:
        return arr.index(get_closest_value(arr, target))-1
    
def hsr(index):
    # Step 1) Get the category
    category_id = food_details['hsr_id'][index]

    # Step 2) Get the required columns
    energy = food_details['Energy, with dietary fibre (in kJ)'][index]
    tsfa = food_details['Total saturated fatty acids (in g)'][index]
    sugars = food_details['Total sugars (in g)'][index]
    sodium = food_details['Sodium (Na) (in mg)'][index]
    protein = food_details['Protein (in g)'][index]
    fibre = food_details['Total dietary fibre (in g)'][index]

    # Step 3) Calculate HSR Baseline Points
    # read tables in as lists
    if food_details['hsr_id'][index] == '1' or food_details['hsr_id'][index] == '1D' or food_details['hsr_id'][index] == '2' or food_details['hsr_id'][index] == '2D':
        hsr_energy = list(hsr_table_1['Energy content (kJ) per 100g or 100mL'])
        hsr_tsfa = list(hsr_table_1['Saturated fatty acids (g) per 100g or 100mL'])
        hsr_sugars = list(hsr_table_1['Total sugars (g) per 100g or 100mL'])
        hsr_sodium = list(hsr_table_1['Sodium (mg) per 100g or 100mL'])
    elif food_details['hsr_id'][index] == '3' or food_details['hsr_id'][index] == '3D': 
        hsr_energy = list(hsr_table_2['Energy content (kJ) per 100g or 100mL'])
        hsr_tsfa = list(hsr_table_2['Saturated fatty acids (g) per 100g or 100mL'])
        hsr_sugars = list(hsr_table_2['Total sugars (g) per 100g or 100mL'])
        hsr_sodium = list(hsr_table_2['Sodium (mg) per 100g or 100mL'])
    else:
        return numpy.nan
    

    # remove nans
    hsr_energy = [x for x in hsr_energy if str(x) != 'nan']
    hsr_tsfa = [x for x in hsr_tsfa if str(x) != 'nan']
    hsr_sugars = [x for x in hsr_sugars if str(x) != 'nan']
    hsr_sodium = [x for x in hsr_sodium if str(x) != 'nan']

    # need some way of quick of getting baseline points based on the table
    baseline_points = []
    baseline_points.append(index_finder(hsr_energy, energy))
    baseline_points.append(index_finder(hsr_tsfa, tsfa))
    baseline_points.append(index_finder(hsr_sugars, sugars))
    baseline_points.append(index_finder(hsr_sodium, sodium))

    # PICK MAX INDEX!!
    baseline_points.sort() 

    # Step 4) Modifying Points (P and F only since we don't have V points)
    # read in p and f tables
    hsr_p = list(hsr_table_3['Protein (g) per 100g or 100mL'])
    hsr_f = list(hsr_table_3['Dietary fibre (g) per 100g or 100mL'])

    p_points = index_finder(hsr_p, protein)
    f_points = index_finder(hsr_f, fibre)

    #Step 5) Calculate Final HSR Score
    final_hsr_score = baseline_points[-1] - p_points - f_points
    hsr = hsr_table_4['Health Star Rating'][list(hsr_table_4[category_id]).index(get_closest_value(list(hsr_table_4[category_id]), final_hsr_score))]
    
    return hsr

ingredient_hsr = []
worked = 0
not_worked = 0

for each in range(len(food_details)):
    try:
        ingredient_hsr.append(hsr(each))
        worked +=1
    except:
        ingredient_hsr.append(numpy.nan)
        not_worked +=1 
        
# need to write back to food_details 
food_details['hsr'] = ingredient_hsr

## Joining to get GHG emissions

In [276]:
food_details = food_details.merge(food_emissions, on = ['emission_id'], how = 'left')

## Filtering down 'food name' and dropping duplicates
At the moment, the food_details table has all the information we need about particular ingredients,however the 'Name' column has a lot of 'duplicates' of ingredients, based on not much needed information.

The goal here is to find by which comma can we split such that the difference in Energy (ie. the range between highest and lowest value), is appropriate.

In [277]:
# find the max number of separations in "Food Name" column
commas = []
for each in food_details["Food Name"]:
    commas.append(len(each.split(',')))

max(commas)

# split food
name_split = food_details["Food Name"].str.split(",", n = 9, expand = True)

food_details["name"] = name_split[0]
food_details["name1"] = name_split[1]
food_details["name2"] = name_split[2]
food_details["name3"] = name_split[3]
food_details["name4"] = name_split[4]
food_details["name5"] = name_split[5]
food_details["name6"] = name_split[6]
food_details["name7"] = name_split[7]
food_details["name8"] = name_split[8]
food_details["name9"] = name_split[9]

# function to calculate range
def mm_diff(x):
           return np.max(x) - np.min(x)
    
names = food_details[['name', 'name1', 'name2', 'name3', 'name4', 'name5', 'name6', 'name7', 'name8', 'name9','Energy, with dietary fibre (in kJ)']]
print('Grouping by name: ' + str(names.groupby('name').agg([mm_diff]).mean()))
print('Grouping by name1: ' + str(names.groupby(['name', 'name1']).agg([mm_diff]).mean()))
print('Grouping by name2: ' + str(names.groupby(['name', 'name1', 'name2']).agg([mm_diff]).mean()))
print('Grouping by name3: ' + str(names.groupby(['name', 'name1', 'name2', 'name3']).agg([mm_diff]).mean()))
print('Grouping by name4: ' + str(names.groupby(['name', 'name1', 'name2', 'name3', 'name4']).agg([mm_diff]).mean()))
print('Grouping by name5: ' + str(names.groupby(['name', 'name1', 'name2', 'name3', 'name4', 'name5']).agg([mm_diff]).mean()))
print('Grouping by name6: ' + str(names.groupby(['name', 'name1', 'name2', 'name3', 'name4', 'name5', 'name6']).agg([mm_diff]).mean()))
print('Grouping by name7: ' + str(names.groupby(['name', 'name1', 'name2', 'name3', 'name4', 'name5', 'name6', 'name7']).agg([mm_diff]).mean()))
print('Grouping by name8: ' + str(names.groupby(['name', 'name1', 'name2', 'name3', 'name4', 'name5', 'name6', 'name7', 'name8']).agg([mm_diff]).mean()))
print('Grouping by name9: ' + str(names.groupby(['name', 'name1', 'name2', 'name3', 'name4', 'name5', 'name6', 'name7', 'name8', 'name9']).agg([mm_diff]).mean()))

Grouping by name: Energy, with dietary fibre (in kJ)  mm_diff    265.771831
dtype: float64
Grouping by name1: Energy, with dietary fibre (in kJ)  mm_diff    104.449184
dtype: float64
Grouping by name2: Energy, with dietary fibre (in kJ)  mm_diff    57.403153
dtype: float64
Grouping by name3: Energy, with dietary fibre (in kJ)  mm_diff    24.626109
dtype: float64
Grouping by name4: Energy, with dietary fibre (in kJ)  mm_diff    22.076253
dtype: float64
Grouping by name5: Energy, with dietary fibre (in kJ)  mm_diff    1.171569
dtype: float64
Grouping by name6: Energy, with dietary fibre (in kJ)  mm_diff    0.0
dtype: float64
Grouping by name7: Energy, with dietary fibre (in kJ)  mm_diff    0.0
dtype: float64
Grouping by name8: Energy, with dietary fibre (in kJ)  mm_diff    0.0
dtype: float64
Grouping by name9: Energy, with dietary fibre (in kJ)  mm_diff    0.0
dtype: float64


In [278]:
# As we can see from the analysis done above, if we only consider 1 comma, then our potential error will be approximately 100kj (which equates to around 20 calories)
# So, we should drop duplicates, based on name and name1 combination
food_details.drop_duplicates(subset = ['name', 'name1'], inplace=True, ignore_index=True)

## Constructing the appropriate data model and posting to the API
We should only send the columns that we need from our dataset:

* name
* name1 (to be called type)
* Description
* Energy, with dietary fibre (in kJ)
* hsr
* hsr_id
* GHG Emissions (kg CO2eq/FU) (Mean)
* emissions_id
* Protein (in g)
* Sodium (Na) (in mg)
* Total sugars (in g)
* Total saturated fatty acids (in g)
* Total dietary fibre (in g)
* Total Fat (in g)
* Available carbohydrate, without sugar alcohols (in g)


In [279]:
# clean up the trailing whitespace
food_details["name1"] = food_details["name1"].str.lstrip()

# Need to clean up the Nones so it can be handled nicely by the API
food_details = food_details.replace({np.nan: None})

# cleanup food_details before we send to API
food_details.rename(columns={'name1': 'ingredient_type',\
                            'Description': 'description',\
                            'Energy, with dietary fibre (in kJ)': 'energy',\
                            'GHG Emissions (kg CO2eq/FU) (Mean)': 'ghg', \
                            'Protein (in g)': 'protein',\
                            'Sodium (Na) (in mg)': 'sodium',\
                            'Total sugars (in g)': 'sugar',\
                            'Total saturated fatty acids (in g)': 'saturated_fat',\
                            'Total dietary fibre (in g)': 'fibre',\
                            'Total Fat (in g)': 'fat',\
                            'Available carbohydrate, without sugar alcohols (in g)': 'carbohydrate'}, inplace=True)

In [287]:
# iterate on each row in the addresses dataset
error = []
for row in food_details.itertuples():
    # form the payload
    ingredient = {
        "name" : row.name, \
        "ingredient_type" : row.ingredient_type, \
        "description" : row.description, \
        "energy" : row.energy, \
        "hsr" : row.hsr, \
        "hsr_id" : row.hsr_id, \
        "ghg": row.ghg, \
        "emission_id" : row.emission_id, \
        "protein" : row.protein, \
        "sodium" : row.sodium, \
        "sugar" : row.sugar, \
        "saturated_fat" : row.saturated_fat, \
        "fibre" : row.fibre, \
        "fat" : row.fat, \
        "carbohydrate" : carbohydrate
    }
    
    # making the API call
    r = requests.post('https://gfood-api.azurewebsites.net/ingredients/add', json=ingredient)
    try:
        if r.status_code != 201:
            error.append(row)
            print("ERROR!")
            print(r.json())
        else:
            print(r.json())
    except:
        continue

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cardamom seed', '_id': '5eb8239f2d568a0043a05370', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb8239f2d568a0043a05370'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Chilli (chili)', '_id': '5eb823a12d568a0043a05371', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823a12d568a0043a05371'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cinnamon', '_id': '5eb823a12d568a0043a05372', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823a12d568a0043a05372'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cloves', '_id': '5eb823a22d568a0043a05373', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823a22d568a0043a05373'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Coriander seed', '_id': '5eb823a22d568a0043a05374', 'r

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Wine', '_id': '5eb823ae2d568a0043a05396', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823ae2d568a0043a05396'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Wine', '_id': '5eb823ae2d568a0043a05397', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823ae2d568a0043a05397'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Wine', '_id': '5eb823af2d568a0043a05398', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823af2d568a0043a05398'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cocoa powder', '_id': '5eb823af2d568a0043a05399', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823af2d568a0043a05399'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Beverage base', '_id': '5eb823af2d568a0043a0539a', 'request': {'type': 

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Water', '_id': '5eb823ba2d568a0043a053bc', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823ba2d568a0043a053bc'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Water', '_id': '5eb823ba2d568a0043a053bd', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823ba2d568a0043a053bd'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Biscuit', '_id': '5eb823bb2d568a0043a053be', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823bb2d568a0043a053be'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Biscuit', '_id': '5eb823bb2d568a0043a053bf', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823bb2d568a0043a053bf'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cone', '_id': '5eb823bb2d568a0043a053c0', 'request': {'type': 'GET', 'u

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cake mix', '_id': '5eb823c62d568a0043a053e2', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823c62d568a0043a053e2'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cake', '_id': '5eb823c72d568a0043a053e3', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823c72d568a0043a053e3'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cake', '_id': '5eb823c72d568a0043a053e4', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823c72d568a0043a053e4'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cake', '_id': '5eb823c72d568a0043a053e5', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823c72d568a0043a053e5'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Cake', '_id': '5eb823c82d568a0043a053e6', 'request': {'type': 'GET', 'url':

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Psyllium', '_id': '5eb823d12d568a0043a05408', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823d12d568a0043a05408'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Quinoa', '_id': '5eb823d22d568a0043a05409', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823d22d568a0043a05409'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Quinoa', '_id': '5eb823d22d568a0043a0540a', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823d22d568a0043a0540a'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Quinoa', '_id': '5eb823d22d568a0043a0540b', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb823d22d568a0043a0540b'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Quinoa', '_id': '5eb823d32d568a0043a0540c', 'request': {'type': 'GET'

...

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Soup', '_id': '5eb824672d568a0043a055f8', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824672d568a0043a055f8'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Soup', '_id': '5eb824672d568a0043a055f9', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824672d568a0043a055f9'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Soup', '_id': '5eb824682d568a0043a055fa', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824682d568a0043a055fa'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Soup', '_id': '5eb824682d568a0043a055fb', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824682d568a0043a055fb'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Bar', '_id': '5eb824692d568a0043a055fc', 'request': {'type': 'GET', 'url': 'htt

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Lolly', '_id': '5eb824732d568a0043a0561e', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824732d568a0043a0561e'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Meringue', '_id': '5eb824732d568a0043a0561f', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824732d568a0043a0561f'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Artichoke', '_id': '5eb824742d568a0043a05620', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824742d568a0043a05620'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Artichoke', '_id': '5eb824742d568a0043a05621', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824742d568a0043a05621'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Artichoke heart', '_id': '5eb824742d568a0043a05622', 'request': 

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Chicory', '_id': '5eb824802d568a0043a05644', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824802d568a0043a05644'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Chilli (chili)', '_id': '5eb824802d568a0043a05645', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824802d568a0043a05645'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Chilli (chili)', '_id': '5eb824802d568a0043a05646', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824802d568a0043a05646'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Chives', '_id': '5eb824812d568a0043a05647', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824812d568a0043a05647'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Choko', '_id': '5eb824812d568a0043a05648', 'request': 

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Potato', '_id': '5eb8248b2d568a0043a0566a', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb8248b2d568a0043a0566a'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Potato', '_id': '5eb8248c2d568a0043a0566b', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb8248c2d568a0043a0566b'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Potato', '_id': '5eb8248c2d568a0043a0566c', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb8248c2d568a0043a0566c'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Potato', '_id': '5eb8248c2d568a0043a0566d', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb8248c2d568a0043a0566d'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Potato', '_id': '5eb8248c2d568a0043a0566e', 'request': {'type': 'GET', 

{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Tomato', '_id': '5eb824972d568a0043a05690', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824972d568a0043a05690'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Tomato', '_id': '5eb824972d568a0043a05691', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824972d568a0043a05691'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Turnip', '_id': '5eb824982d568a0043a05692', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824982d568a0043a05692'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Vine leaf', '_id': '5eb824982d568a0043a05693', 'request': {'type': 'GET', 'url': 'https://gfood-api.azurewebsites.net/ingredients/5eb824982d568a0043a05693'}}}
{'message': 'Ingredient Added', 'addedIngredient': {'name': 'Water chestnut', '_id': '5eb824982d568a0043a05694', 'request': {'typ